# Main Tests
### <font color='5E5D5D'> Descripción </font>

<i><font color='C9614B'> En esta notebook se encuentra la mayor parte de las pruebas realizadas en lo que refiere a entrenamiento de agentes DQN y NFQ sin ventanas de rollout (Experimentos 1, 3 y 4). Tener en cuenta que la configuración que se presenta aquí, representa únicamente una de las evaluadas.</font></i><br>

***

## Configuración

In [ ]:
PLAY_VIDEO = False

#### Librerías 

In [ ]:
# Librerías del sistema que son prerequsito: descomentar estas líneas únicamente si se necesita su instalaciòn
#! pip install gym pyvirtualdisplay > /dev/null 2>&1
#! apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

# Modelos
from models.models import DQNModel, NFQModel

# Agentes
from agents.dqn_agent import DQNAgent
from agents.nfq_agent import NFQAgent

# Utilitarios
from utils.utils_func import process_state, save_dataset, load_dataset
from utils.visualization import plot_results, wrap_env, show_video

# Gym
import gym

# PyTorch
import torch

# Misc
import numpy as np
import random

# Gráficos
import matplotlib.pyplot as plt

## Tests para los Experimentos 1, 3 y 4

### Deep Q-Learning

Entrenamiento de agentes DQN, utilizando el ambiente ``MountainCar-v0``.

#### Mountain Car

In [ ]:
# Variables globales para el entrenamiento
BUFFER_SIZE = 2000
GAMMA = 0.99
NUM_EPISODES = 1000
MAX_STEPS = 200

BATCH_SIZE = 64
LEARNING_RATE = 0.001

# Generación de datasets (RAND, ESA, EMA)
NUM_SAMPLES = 1000
NUM_RUNS = 1
DATASET_ACTION_TYPE='greedy'

De manera de evitar -en cierta medida- particularidades en las ejecuciones, se realizan ``NUM_RUNS`` entrenamientos. Para cada uno de ellos, se genera un dataset de experiencia y se agrega al dataset final que se utilizará para entrenar NFQ.

In [ ]:
# Arrays para los resultados finales
rewards_mc_dqn, steps_mc_dqn = [], []
agent_mc_dqn = None

# Modelo para el agente DQN
model_dqn = None

# Dataset para almacenar la experiencia
dataset = []

# Lista para almacenar las trayectorias (para el cálculo de RIS)
trajectories = []

# Seed inicial (puede tomar cualquier valor deseado)
num_seed = np.random.randint(0, 1000)

for _run in range(NUM_RUNS):
  # Info de la corrida
  print(f"\nRun #{_run+1} | Seed: {num_seed}")
  print("********************************************************")

  # Ambiente
  env = gym.make("MountainCar-v0")

  # Configuraciòn del seed
  env.seed(num_seed)
  random.seed(num_seed)
  np.random.seed(num_seed)
  torch.manual_seed(num_seed)
  torch.backends.cudnn.deterministic = True
  
  # Creación del modelo
  model_dqn = DQNModel(2, env.action_space.n)

  # Creación del agente
  agent_mc_dqn = DQNAgent(env, model_dqn, process_state, 
                          BUFFER_SIZE, BATCH_SIZE, LEARNING_RATE, GAMMA, 
                          epsilon_i=0.99, epsilon_f=0.1, epsilon_anneal_time=1000)

  # Entrenamiento del agente
  rewards, steps_per_episode = agent_mc_dqn.train(NUM_EPISODES, MAX_STEPS)
  
  # Guardar resultados
  rewards_mc_dqn.append(rewards)
  steps_mc_dqn.append(steps_per_episode)

  # Generar dataset(s) utilizando agente(s) entrenado(s)
  env = gym.make("MountainCar-v0")
  experience, traject = agent_mc_dqn.generate_dataset(env, action_type=DATASET_ACTION_TYPE, epsilon=.1, num_samples=NUM_SAMPLES, max_steps=MAX_STEPS)
  dataset.extend(experience)
  trajectories.extend(traject)

  # Incrementar el seed
  num_seed += 1

# Promedio de los resultados (por corrida)
rewards_mc_dqn = np.mean(rewards_mc_dqn, axis=0)
steps_mc_dqn = np.mean(steps_mc_dqn, axis=0)

In [ ]:
# Gráfico "suavizado"
plot_results(NUM_EPISODES, rewards_mc_dqn, steps_mc_dqn)

In [ ]:
if PLAY_VIDEO:
    # Verificar aprendizaje a través del video
    wrapped_env = wrap_env(gym.make("MountainCar-v0"))

    agent_mc_dqn.record_test_episode(wrapped_env)

In [ ]:
# Guardar el dataset de experiencia en formato pickle
filename = 'datasets/dataset_sample.pkl'
save_dataset(filename, dataset)

In [ ]:
# Guardar el dataset de trayectorias en formato pickle
filename = 'datasets/trajectories_sample.pkl'
save_dataset(filename, trajectories)

### Neural Fitted Q-Iteration

Entrenamiento de agentes NFQ, utilizando el ambiente ``MountainCar-v0``.

#### Mountain Car

In [ ]:
# Cargar el dataset de experiencia desde archivo pickle
filename = 'datasets/dataset_sample.pkl'
dataset = load_dataset(filename)

In [ ]:
# Cargar el dataset de trayectorias desde archivo pickle
filename = 'datasets/trajectories_sample.pkl'
trajectories = load_dataset(filename)

In [ ]:
# Variables globales para el entrenamiento
GAMMA = 0.99
NUM_EPISODES = 2000
MAX_STEPS = 200
BATCH_SIZE = 64
LEARNING_RATE = 0.001
EARLY_STOPPING = True
EARLY_STOPPING_PATIENCE = 100

# Variables para RIS
N_ACTIONS = 3
HORIZON = 200

# Variables para el testing
IS_TEST = True
TEST_RUN_TRIALS = 100

In [ ]:
# NFQ
# Listas para los resultados finales
rewards_mc_nfq, steps_mc_nfq = [], []

# Ambiente
env = gym.make("MountainCar-v0")

# Creación del modelo
model_nfq = NFQModel(2, env.action_space.n)

# Creación del agente
agent_mc_nfq = NFQAgent(env, model_nfq, process_state, 
                        BATCH_SIZE, LEARNING_RATE, GAMMA,
                        dataset, trajectories, 
                        N_ACTIONS, HORIZON)

In [ ]:
# Entrenamiento del agente
rewards_mc_nfq, _, ris_mc_nfq = agent_mc_nfq.train_from_dataset(NUM_EPISODES, is_test=IS_TEST, test_run_trials=TEST_RUN_TRIALS, early_stopping=EARLY_STOPPING, es_patience=EARLY_STOPPING_PATIENCE)

In [ ]:
# Plot de RIS
# El eje 'x' tiene que tener dimensiones de (episode number + 1)
plt.plot(range(102), ris_mc_nfq)
plt.title("Episode RIS")
plt.xlabel("Episode Number")
plt.ylabel("Value")
plt.show()

In [ ]:
# Plot de rewards de test_run
plt.plot(range(200), [item for sublist in rewards_mc_nfq for item in sublist])
plt.title("Test Run Rewards")
plt.xlabel("Episode Number")
plt.ylabel("Reward")
plt.show()

In [ ]:
if PLAY_VIDEO:
    # Verificar aprendizaje a través del video
    wrapped_env = wrap_env(gym.make("MountainCar-v0"))

    agent_mc_nfq.record_test_episode(wrapped_env)